In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import (
    SimpleRNN,
    Input,
    Dense
)
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
EPOCHS = 100
BATCH_SIZE = 64

In [3]:
df = pd.read_csv("Electric_Production.csv", parse_dates=[0])
df

,DATE,IPG2211A2N
0,1985-01-01,72.5052
1,1985-02-01,70.6720
2,1985-03-01,62.4502
3,1985-04-01,57.4714
4,1985-05-01,55.3151
...,...,...
392,2017-09-01,98.6154
393,2017-10-01,93.6137
394,2017-11-01,97.3359
395,2017-12-01,114.7212


In [4]:
df = df.set_index('DATE')

In [5]:
df.head()

,IPG2211A2N
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


In [6]:
seq_len = 3

In [7]:
from numpy.lib.stride_tricks import sliding_window_view

In [8]:
all_data_transformed = sliding_window_view(df.iloc[:, 0].values, window_shape=seq_len+1)
all_data_transformed

array([[ 72.5052,  70.672 ,  62.4502,  57.4714],
       [ 70.672 ,  62.4502,  57.4714,  55.3151],
       [ 62.4502,  57.4714,  55.3151,  58.0904],
       ...,
       [108.9312,  98.6154,  93.6137,  97.3359],
       [ 98.6154,  93.6137,  97.3359, 114.7212],
       [ 93.6137,  97.3359, 114.7212, 129.4048]])

In [9]:
X = all_data_transformed[:, :seq_len]
X

array([[ 72.5052,  70.672 ,  62.4502],
       [ 70.672 ,  62.4502,  57.4714],
       [ 62.4502,  57.4714,  55.3151],
       ...,
       [108.9312,  98.6154,  93.6137],
       [ 98.6154,  93.6137,  97.3359],
       [ 93.6137,  97.3359, 114.7212]])

In [10]:
y = all_data_transformed[:, seq_len]
y

array([ 57.4714,  55.3151,  58.0904,  62.6202,  63.2485,  60.5846,
        56.3154,  58.0005,  68.7145,  73.3057,  67.9869,  62.2221,
        57.0329,  55.8137,  59.9005,  65.7655,  64.4816,  61.0005,
        57.5322,  59.3417,  68.1354,  73.8152,  70.062 ,  65.61  ,
        60.1586,  58.8734,  63.8918,  68.8694,  70.0669,  64.1151,
        60.3789,  62.4643,  70.5777,  79.8703,  76.1622,  70.2928,
        63.2384,  61.4065,  67.1097,  72.9816,  75.7655,  67.5152,
        63.2832,  65.1078,  73.8631,  77.9188,  76.6822,  73.3523,
        65.1081,  63.6892,  68.4722,  74.0301,  75.0448,  69.3053,
        65.8735,  69.0706,  84.1949,  84.3598,  77.1726,  73.1964,
        67.2781,  65.8218,  71.4654,  76.614 ,  77.1052,  73.061 ,
        67.4365,  68.5665,  77.6839,  86.0214,  77.5573,  73.365 ,
        67.15  ,  68.8162,  74.8448,  80.0928,  79.1606,  73.5743,
        68.7538,  72.5166,  79.4894,  85.2855,  80.1643,  74.5275,
        69.6441,  67.1784,  71.2078,  77.5081,  76.5374,  72.3

In [11]:
n_features = 1
n_outputs = 1
rnn_units_count = 30

In [12]:
X = X.reshape(X.shape[0], seq_len, n_features)
X

array([[[ 72.5052],
        [ 70.672 ],
        [ 62.4502]],

       [[ 70.672 ],
        [ 62.4502],
        [ 57.4714]],

       [[ 62.4502],
        [ 57.4714],
        [ 55.3151]],

       ...,

       [[108.9312],
        [ 98.6154],
        [ 93.6137]],

       [[ 98.6154],
        [ 93.6137],
        [ 97.3359]],

       [[ 93.6137],
        [ 97.3359],
        [114.7212]]])

In [13]:
model = Sequential()
model.add(Input(shape=(seq_len, n_features)))
model.add(SimpleRNN(rnn_units_count, activation='linear'))
model.add(Dense(n_outputs))

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 30)                  │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 991 (3.87 KB)

 Trainable params: 991 (3.87 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [16]:
model.fit(
    X, y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1
)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 4977.1460 - val_loss: 1121.8436
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 451.6411 - val_loss: 588.2161
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 546.2262 - val_loss: 923.5905
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 522.6686 - val_loss: 303.5325
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 173.5581 - val_loss: 317.5258
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 219.3595 - val_loss: 345.8983
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 197.3333 - val_loss: 247.3166
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 135.8304 - val_loss: 248.3964
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 151.6470 - val_loss: 246.6248
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 136.6512 - val_loss: 231.5761
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 140.9430 - val_loss: 235.5291
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 73.0515 - val_loss: 125.0209
Epoch 69/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 70.0459 - val_loss: 124.0274
Epoch 70/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 70.7793 - val_loss: 122.7067
Epoch 71/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 71.3548 - val_loss: 122.1770
Epoch 72/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 69.3084 - val_loss: 120.3911
Epoch 73/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 68.6053 - val_loss: 120.1297
Epoch 74/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 65.8729 - val_loss: 118.2886
Epoch 75/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 67.0368 - val_loss: 117.5999
Epoch 76/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 67.9285 - val_loss: 116.2589
Epoch 77/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 66.2240 - val_loss: 117.3410
Epoch 78/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 66.2672 - val_loss: 114.1542
Epoch 79/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss